In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('data hw2.csv', header = None)

data.head()

data = np.array(data)


In [2]:
def entropy(target = data[:,-1]):
    '''
    Calculate the entropy of a specific column in the data set
    1) accepts a vector
    
    returns the entropy
    '''
    entropy = 0.0
    values, counts = np.unique(target,return_counts = True)
    for i in range(0,len(values)):
        entropy = np.sum((-1.0*counts[i]/sum(counts))*np.log2(1.0*counts[i]/sum(counts)) ) + entropy
    return entropy # , values, counts


entropy()

0.8812908992306927

In [5]:
"""
Information Gain 
"""

def IG(data, feature, target = -1 ):
    """
    Calculate the information gain of a dataset
    Params:
    1) data = The dataset 
    2) feature = the column index for which the information gain will be calculated
    3) target = the column index of the information
    
    Categorical
        If the feature only has two unique values the function will return the IG and a string 'cat'
    
    Non- Categorical 
        If there are more than two unique features the function will return the maximum IG, the value at which the max IG occurs and a list of the weighted average Entropy
    
    """    
    #data = np.array(data)
    #Calculate the entropy of the total dataset
    target_entropy = entropy(data[:,target])
    
    ##Calculate the entropy of the dataset
    
    #Calculate the values and the corresponding counts for the split attribute 
    values,counts= np.unique(data[:, feature],return_counts=True)
    #print(values, counts)
    #Calculate the weighted entropy for categorical values
    w_entropy = 0.0
    if len(values) ==2:
        
        for i in range(0,len(values)): 
            w_entropy = (1.0*counts[i]/sum(counts))*entropy(data[:,target][data[:,feature] == values[i]]) + w_entropy
        #Calculate the information gain
        IG = target_entropy - w_entropy
        #print(target_entropy, w_entropy, target_entropy - w_entropy )
        return IG, str('cat')
    
    # return the weighted entropys for noncategorical values
    else:    
        l =[]
        for i in range(1,len(values)): 
                #print counts[:i]
                #print counts[i:]
                #print values[i-1]
                
            l.append([((1.0*sum(counts[:i])/sum(counts))*entropy(data[:,target][data[:,feature] <= values[i-1]]) + (1.0*sum(counts[i:])/sum(counts))*entropy(data[:,target][data[:,feature] > values[i-1]])), values[i-1]]) 
        w_entropy = min(l)


        #Calculate the information gain
        split = w_entropy[1]
        IG = target_entropy - w_entropy[0]
        #print(target_entropy, w_entropy, target_entropy - w_entropy )
        return IG, split, l

IG(data,4,-1)


(0.02295208771298607,
 6.0,
 [[0.865733522695209, 0.0],
  [0.8811032678472634, 1.0],
  [0.8767253119276491, 2.0],
  [0.8799161341423124, 3.0],
  [0.8736695709744837, 4.0],
  [0.8785317651150599, 5.0],
  [0.8583388115177066, 6.0]])

In [7]:

'''
Start Gini
'''


def G(data, feature =0, target = -1):
    """
    Calculate the information gain of a dataset
    Params:
    1) data = The dataset 
    2) feature = the column index for which the information gain will be calculated
    3) target = the column index of the information
    
    Categorical
        If the feature only has two unique values the function will return the IG and a string 'cat'
    
    Non- Categorical 
        If there are more than two unique features the function will return the maximum IG, the value at which the max IG occurs and a list of the weighted average Entropy
    
    """ 
    gini = 0
    values,counts= np.unique(data[:, feature],return_counts=True)
    if len(values) ==2:
        for i in [0,1]:
            gini = gini + (1.0*counts[i]/sum(counts))* (1-(sum(data[:,target][data[:,feature] == values[i]])/len(data[:,target][data[:,feature] == values[i]]))**2)
        return gini, [i]

    else:
        l = []
        for i in range(1,len(values)):
            l.append(((1.0*sum(counts[:i])/sum(counts))* (1-(sum(data[:,target][data[:,feature] <= values[i-1]])/len(data[:,target][data[:,feature] <= values[i-1]]))**2)
             +(1.0*sum(counts[i:])/sum(counts))* (1-(sum(data[:,target][data[:,feature] > values[i-1]])/len(data[:,target][data[:,feature] > values[i-1]]))**2), values[i-1]))
        gini = min(l)    
        return gini
G(data,6)

(0.9010989010989011, 1.0)

In [8]:

'''
Start CART
'''
def CART(data, feature = 0, target = -1):
    """
    Calculate the information gain of a dataset
    Params:
    1) data = The dataset 
    2) feature = the column index for which the information gain will be calculated
    3) target = the column index of the information
    

    """ 
    
    
    values,counts= np.unique(data[:, feature],return_counts=True)
    
    CART = []
    for i in range(1,len(values)):
        cy0 = sum(data[:,target][data[:,feature] <= values[i-1]])/len(data[:,target][data[:,feature] <= values[i-1]])
        cy1 = sum(data[:,target][data[:,feature] > values[i-1]])/len(data[:,target][data[:,feature] > values[i-1]])
        
        CART.append([abs(cy0 - cy1)*2.0*(1.0*len(data[:,target][data[:,feature] > values[i-1]])*len(data[:,target][data[:,feature] <= values[i-1]])) / (len(data)**2), values[i-1]])
    return max(CART)


CART(data, 5)



[0.21, 4.0]

In [11]:

'''
{"IG","G","CART"}
bestsplit(D, criterion)
'''

def bestsplit(data, criterion = "IG"):
   
    ''' IG '''    
    if criterion == "IG":
        ig = []
        for i in range(0,10):
            ig.append([IG(data,i, -1),i])
        feature =  max(ig)[1]
        x,value,z = max(ig)[0]
        return (feature, value)
    
    
    ''' GINI '''
    if criterion == "G":
        g = []
        for i in range(0,10):
            g.append( (G(data,i, -1),i))

        feature = min(g)[1]
        x,value = min(g)[0]
        return (feature, value)


    '''CART'''
    if criterion == "CART":
        cart = []
        for i in range(0,10):
            cart.append((CART(data,i, -1) , i))
        feature =  max(cart)[1]
        x,value = max(cart)[0]   
        return (feature, value)
        
    else:
        print(" criterion should equal one of the following: IG,G,CART" )

bestsplit(data, criterion = "CART")

(2, 0.0)

In [12]:

test_data = pd.read_csv('test data hw2.csv', header = None)
test_data.head()
test_data = np.array(test_data)


In [13]:


'''
classifyIG(train, test) 
'''

def classifyIG(train = data, test = test_data):
    '''
    Make predictions based on the information gain classifier
    1) the training data
    2) the test data
    3) assumes the target column is -1
    function calls best split, then determines which binary classifier 
    to assign to the greater than, or less than. finally it 
    
    '''
    f, v =  bestsplit(data, criterion = "IG")
    
    high = sum(data[:,-1][data[:,f] > v]) / len(data[:,-1][data[:,f] > v])
    low = sum(data[:,-1][data[:,f] <= v]) / len(data[:,-1][data[:,f] <= v])
    
    if high > low:
        high = 1
        low = 0
    else:
        high = 0
        low = 1
    
    pred = []
    
    for i in range(len(test_data)):
        if test_data[:,f][i] > v:
            pred.append([high, test_data[:,-1][i], high == test_data[:,-1][i] ])
        if test_data[:,f][i] <= v:
            pred.append([low, test_data[:,-1][i], low == test_data[:,-1][i]] )
    return pred
classifyIG()



[[0, 0.0, True],
 [0, 0.0, True],
 [0, 0.0, True],
 [1, 0.0, False],
 [1, 1.0, True],
 [1, 1.0, True],
 [1, 0.0, False],
 [1, 1.0, True],
 [0, 0.0, True],
 [0, 0.0, True]]

In [14]:

'''
classifyG(train, test),
'''


def classifyG(train = data, test = test_data):
    '''
    Make predictions based on the information gain classifier
    1) the training data
    2) the test data
    3) assumes the target column is -1
    function calls best split, then determines which binary classifier 
    to assign to the greater than, or less than. finally it 
    
    '''
    f, v =  bestsplit(data, criterion = "G")
    
    high = sum(data[:,-1][data[:,f] > v]) / len(data[:,-1][data[:,f] > v])
    low = sum(data[:,-1][data[:,f] <= v]) / len(data[:,-1][data[:,f] <= v])
    
    if high > low:
        high = 1
        low = 0
    else:
        high = 0
        low = 1
    
    pred = []
    
    for i in range(len(test_data)):
        if test_data[:,f][i] > v:
            pred.append([high, test_data[:,-1][i], high == test_data[:,-1][i] ])
        if test_data[:,f][i] <= v:
            pred.append([low, test_data[:,-1][i], low == test_data[:,-1][i]] )
    return pred

classifyG()

[[0, 0.0, True],
 [0, 0.0, True],
 [0, 0.0, True],
 [0, 0.0, True],
 [1, 1.0, True],
 [1, 1.0, True],
 [0, 0.0, True],
 [0, 1.0, False],
 [0, 0.0, True],
 [0, 0.0, True]]

In [15]:

'''
classifyCART(train, test)
'''
def classifyCART(train = data, test = test_data):
    '''
    Make predictions based on the information gain classifier
    1) the training data
    2) the test data
    3) assumes the target column is -1
    function calls best split, then determines which binary classifier 
    to assign to the greater than, or less than. finally it 
    
    '''
    f, v =  bestsplit(data, criterion = "CART")
    
    high = sum(data[:,-1][data[:,f] > v]) / len(data[:,-1][data[:,f] > v])
    low = sum(data[:,-1][data[:,f] <= v]) / len(data[:,-1][data[:,f] <= v])
    
    if high > low:
        high = 1
        low = 0
    else:
        high = 0
        low = 1
    
    pred = []
    
    for i in range(len(test_data)):
        if test_data[:,f][i] > v:
            pred.append([high, test_data[:,-1][i], high == test_data[:,-1][i] ])
        if test_data[:,f][i] <= v:
            pred.append([low, test_data[:,-1][i], low == test_data[:,-1][i]] )
    return pred

classifyCART()


[[0, 0.0, True],
 [0, 0.0, True],
 [0, 0.0, True],
 [0, 0.0, True],
 [1, 1.0, True],
 [1, 1.0, True],
 [0, 0.0, True],
 [1, 1.0, True],
 [0, 0.0, True],
 [0, 0.0, True]]